In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "af", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "af", split="test", use_auth_token=True)

print(common_voice)


/Users/kenneth.hamilton/Desktop/CommonVoice_Whisper_Trainer/venv/lib/python3.11/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Reading metadata...: 82it [00:00, 116508.44it/s]mples/s]
Generating train split: 82 examples [00:00, 2918.84 examples/s]
Reading metadata...: 54it [00:00, 64972.01it/s] ? examples/s]
Generating validation split: 54 examples [00:00, 3472.64 examples/s]
Reading metadata...: 62it [00:00, 93542.03it/s]mples/s]
Generating test split: 62 examples [00:00, 1824.93 examples/s]
Reading metadata...: 15it [00:00, 120065.95it/s]mples/s]
Generating other split: 15 examples [00:00, 2446.99 examples/s]
Reading metadata...: 192it [00:00, 176602.27it/s]? examples/s]
Generating invalidated split: 192 examples [00:00, 10193.75 examples/s]
Reading metadata...: 198it [00:00, 228402.69it/s]ex

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 136
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 62
    })
})


In [5]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [6]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Afrikaans", task="transcribe")

In [9]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Vertraging in die aanstelling van diensverskaffers maak dit moeilik om projekte betyds te voltooi.
Decoded w/ special:    <|startoftranscript|><|af|><|transcribe|><|notimestamps|>Vertraging in die aanstelling van diensverskaffers maak dit moeilik om projekte betyds te voltooi.<|endoftext|>
Decoded w/out special: Vertraging in die aanstelling van diensverskaffers maak dit moeilik om projekte betyds te voltooi.
Are equal:             True


In [11]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Afrikaans", task="transcribe")

In [12]:
print(common_voice["train"][0])

{'audio': {'path': '/Users/kenneth.hamilton/.cache/huggingface/datasets/downloads/extracted/67c61da9ebe5c136a40751310fe48499793029d48bd8253f5be1fbb64cc5805a/af_train_0/common_voice_af_39597042.mp3', 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       3.93024220e-06, 5.23015660e-06, 3.42527369e-06]), 'sampling_rate': 48000}, 'sentence': 'Vertraging in die aanstelling van diensverskaffers maak dit moeilik om projekte betyds te voltooi.', 'variant': ''}


In [13]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [14]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [15]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)

Map (num_proc=4): 100%|██████████| 62/62 [00:00<00:00, 222.09 examples/s]


In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Error while downloading from https://cdn-lfs.huggingface.co/repos/ca/81/ca8175f7fe33afb926eb59ad7c1664aaff834559c34d93520c9d26aff4b1e408/1d7734884874f1a1513ed9aa760a4f8e97aaa02fd6d93a3a85d27b2ae9ca596b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1725453235&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTQ1MzIzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jYS84MS9jYTgxNzVmN2ZlMzNhZmI5MjZlYjU5YWQ3YzE2NjRhYWZmODM0NTU5YzM0ZDkzNTIwYzlkMjZhZmY0YjFlNDA4LzFkNzczNDg4NDg3NGYxYTE1MTNlZDlhYTc2MGE0ZjhlOTdhYWEwMmZkNmQ5M2EzYTg1ZDI3YjJhZTljYTU5NmI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=e3pefx5wvbZuBAe20V157X4h4iWCiSwKU%7E9qIMLNVcpzRFTFLfqAbU1kzAZQaEmxcED3E1j4%7EEtV0VsIlbH6lKWS82ZZTRFrc5ndx4mMecNZR4Oed03vU5JTxubmJWkpPFSdaeZW4vRpQaBCtFModQmc-vCFpj4qfXeYHbqp824f-Jqck02ZLr-vq%7EQwLft%7E037sPCWIB-uV-c1xlo3oslu7ZarfSVNY37Zy5P3PbBmt2e

In [18]:
model.generation_config.language = "afrikaans"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [19]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [20]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [21]:
import evaluate

metric = evaluate.load("wer")

In [22]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [29]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./afrikaaner-whisper",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


/Users/kenneth.hamilton/Desktop/CommonVoice_Whisper_Trainer/venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

ValueError: fp16 mixed precision requires a GPU (not 'mps').